### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [3]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 45.311666
2 		 16.422026
3 		 4.354258
4 		 2.763284
5 		 2.622467
6 		 2.581300
7 		 2.548310
8 		 2.513893
9 		 2.476235
10 		 2.434754


In [4]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.91658705  8.93710537 10.62958907 10.18787445]
 [ 8.74283108  7.44551825  9.17092452  8.64685015]
 [ 7.87060682  6.98784122  8.09709759  7.86313342]
 [ 8.90884188  8.53049562 10.32327154  9.57087189]
 [ 9.14125045  7.10492112  8.77857155  8.54680945]
 [ 6.47118355  6.29668238  7.30890165  6.82463427]
 [ 9.28025007  8.00792073  9.7774023   9.08818091]
 [ 8.74562298  6.98681809  8.24530059  8.31966214]
 [ 8.48530151  7.45198624  8.8133204   8.5345559 ]
 [ 8.20016488  6.55981693  7.82663134  7.67384597]
 [ 8.81639242  7.9267448   9.33708358  8.79372335]
 [ 9.25803594  8.57259776 10.29564196  9.44100474]
 [ 9.08334321  8.09742481  9.75284161  9.35327129]
 [ 6.07517076  6.25399991  7.04496351  6.5886066 ]
 [10.13271995  8.33606101  9.99971793  9.82257296]
 [ 9.41283006  8.75007124 10.2439173   9.7807656 ]
 [ 8.28507219  7.05252422  8.85240846  8.28704849]
 [ 7.73748696  6.13627274  6.9128057   7.24076673]
 [ 7.8281214   7.40855612  9.07701251  8.39971065]
 [ 8.51420474  6.93181497  8.33

**The predicted ratings from the dot product are already starting to look a lot like the original data values even after only 10 iterations. You can see some extreme low values that are not captured well yet. The 5 in the second to last row in the first column is predicted as an 8, and the 4 in the second row and second column is predicted to be a 7. Clearly the model is not done learning, but things are looking good.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [5]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 44.407420
2 		 16.030360
3 		 4.416249
4 		 2.850842
5 		 2.706236
6 		 2.675186
7 		 2.656762
8 		 2.638681
9 		 2.618643
10 		 2.596002
11 		 2.570319
12 		 2.541120
13 		 2.507867
14 		 2.469972
15 		 2.426807
16 		 2.377730
17 		 2.322105
18 		 2.259344
19 		 2.188955
20 		 2.110609
21 		 2.024219
22 		 1.930021
23 		 1.828661
24 		 1.721245
25 		 1.609358
26 		 1.495016
27 		 1.380541
28 		 1.268370
29 		 1.160819
30 		 1.059847
31 		 0.966869
32 		 0.882662
33 		 0.807366
34 		 0.740582
35 		 0.681521
36 		 0.629172
37 		 0.582450
38 		 0.540313
39 		 0.501832
40 		 0.466232
41 		 0.432899
42 		 0.401373
43 		 0.371332
44 		 0.342568
45 		 0.314969
46 		 0.288495
47 		 0.263156
48 		 0.238999
49 		 0.216087
50 		 0.194490
51 		 0.174272
52 		 0.155482
53 		 0.138149
54 		 0.122281
55 		 0.107861
56 		 0.094849
57 		 0.083186
58 		 0.072796
59 		 0.063593
60 		 0.055481
61 		 0.048362
62 		 0.042138
63 		 0.036711
64 		

In [6]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.99999348  9.99999748  9.99999891 10.00000596]
 [ 9.9999992   3.99999977  8.99999995 10.00000075]
 [ 8.00000303  9.00000136 10.00000068  4.99999717]
 [ 8.99999246  7.99999714  9.99999879 10.00000693]
 [10.00000542  5.0000023   9.00000112  8.99999503]
 [ 5.99999578  3.99999841  9.99999933  6.00000388]
 [ 9.00000351  8.00000155 10.00000079  8.99999677]
 [10.00000091  5.0000004   9.00000021  7.99999918]
 [ 7.00001991  8.000008   10.00000361  7.99998159]
 [ 9.00000781  5.00000294  9.00000121  6.99999274]
 [ 8.9999974   7.99999873  9.99999927  8.00000229]
 [ 8.99999916  9.99999949  9.99999964  9.00000066]
 [ 9.99999942  8.99999962  9.99999971  8.00000042]
 [ 5.00000196  8.00000073  5.00000028  7.99999812]
 [ 9.99999619  7.99999834  9.99999914 10.00000342]
 [ 9.00000352  9.00000132 10.00000052  9.99999665]
 [ 9.00000353  8.00000131  8.00000049  7.99999664]
 [10.00000005  7.99999998  0.99999994  9.99999988]
 [ 4.99999938  5.99999961  9.99999972 10.00000047]
 [ 7.99998474  6.99999383  9.99

**In this case, we were able to completely reconstruct the item-movie matrix to obtain an essentially 0 mean squared error.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [7]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [8]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 46.228008
2 		 17.722437
3 		 4.779263
4 		 2.839175
5 		 2.632101
6 		 2.567127
7 		 2.514758
8 		 2.459137
9 		 2.396833
10 		 2.326669
11 		 2.248037
12 		 2.160641
13 		 2.064537
14 		 1.960221
15 		 1.848702
16 		 1.731542
17 		 1.610823
18 		 1.489050
19 		 1.368958
20 		 1.253275
21 		 1.144459
22 		 1.044466
23 		 0.954603
24 		 0.875471
25 		 0.807023
26 		 0.748686
27 		 0.699524
28 		 0.658404
29 		 0.624126
30 		 0.595532
31 		 0.571556
32 		 0.551265
33 		 0.533860
34 		 0.518668
35 		 0.505135
36 		 0.492803
37 		 0.481297
38 		 0.470309
39 		 0.459587
40 		 0.448922
41 		 0.438142
42 		 0.427103
43 		 0.415691
44 		 0.403810
45 		 0.391391
46 		 0.378379
47 		 0.364745
48 		 0.350475
49 		 0.335581
50 		 0.320093
51 		 0.304065
52 		 0.287569
53 		 0.270700
54 		 0.253571
55 		 0.236309
56 		 0.219054
57 		 0.201954
58 		 0.185156
59 		 0.168807
60 		 0.153042
61 		 0.137983
62 		 0.123734
63 		 0.110377
64 		

In [9]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.070374275741957:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [10]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 23.540757
2 		 10.783983
3 		 7.380735
4 		 5.700237
5 		 4.645414
6 		 3.905452
7 		 3.352051
8 		 2.921413
9 		 2.577278
10 		 2.297009
11 		 2.065534
12 		 1.872317
13 		 1.709677
14 		 1.571799
15 		 1.454149
16 		 1.353112
17 		 1.265784
18 		 1.189829
19 		 1.123366
20 		 1.064884


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [12]:
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [13]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
